In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

from torch import nn

In [ ]:
TARGET_COLUMN = 'target'
METAINFO_COLUMNS = ['stage', 'move_count', 'weight']
METAINFO_COLUMNS_COUNT = len(METAINFO_COLUMNS)
STAGE_INDEX = 0
MOVE_COUNT_INDEX = 1
WEIGHT_INDEX = 2
TARGET_INDEX = -1

STARTPOS_STAGE = 24
INPUT_LAYER_SIZE = 64 * 12

BATCH_SIZE = 131072
TRAIN_SIZE = 0.9

In [ ]:
dataset = np.zeros((0, INPUT_LAYER_SIZE + METAINFO_COLUMNS_COUNT + 1), np.int8)
for chunk in tqdm(pd.read_csv('/home/wind-eagle/quirky_data/dataset/01.csv', chunksize=125000)):
    dataset = np.append(dataset, chunk.values.astype(np.int8), axis=0)

np.random.shuffle(dataset)
train_data_size = int(dataset.shape[0] * TRAIN_SIZE)

In [ ]:
class ClippedReLU(nn.Module):
    def __init__(self, clip_value: float):
        super(ClippedReLU, self).__init__()
        self.clip_value = clip_value

    def forward(self, x):
        return torch.clamp(x.relu(), max=self.clip_value)

In [ ]:
class QNNE(nn.Module):
    def __init__(self):
        super(QNNE, self).__init__()

        self.opening = nn.Sequential()
        
        self.opening.add_module('linear1', nn.Linear(INPUT_LAYER_SIZE, 16))
        self.opening.add_module('clipped_relu1', ClippedReLU(1.0))
        self.opening.add_module('linear2', nn.Linear(8, 8))
        self.opening.add_module('clipped_relu2', ClippedReLU(1.0))
        self.opening.add_module('linear3', nn.Linear(8, 1))
        self.opening.add_module('sigmoid', nn.Sigmoid())

    def forward(self, x):
        return self.opening(x[:, METAINFO_COLUMNS_COUNT:TARGET_INDEX])[:, 0]

In [ ]:
def get_loss(model, X, y, w):
    y = (y + 1.0) / 2
    y_pred = model(X)
    assert y_pred.dim() == 1
    loss = torch.sum(w * ((y - y_pred) ** 2)) / torch.sum(w)
    return loss

In [ ]:
model = QNNE()
opt = torch.optim.Adam(model.parameters(), lr=0.01)

history = []
train_loss = np.inf
test_loss = np.inf

progress = tqdm(np.arange(12000))
for i in progress:
    indices = np.random.randint(0, train_data_size, BATCH_SIZE)
    X_batch = torch.tensor(dataset[indices], dtype=torch.float32)
    y_batch = torch.tensor(dataset[indices, TARGET_INDEX], dtype=torch.float32)
    w_batch = torch.tensor(dataset[indices, WEIGHT_INDEX], dtype=torch.float32)
    
    loss = get_loss(model, X_batch, y_batch, w_batch)

    loss.backward()
    
    opt.step()
    opt.zero_grad()

    history.append(loss.data.numpy())

    if len(history) % 50 == 1:
        X_batch = torch.tensor(dataset[train_data_size:], dtype=torch.float32)
        y_batch = torch.tensor(dataset[train_data_size:, TARGET_INDEX], dtype=torch.float32)
        w_batch = torch.tensor(dataset[train_data_size:, WEIGHT_INDEX], dtype=torch.float32)
        test_loss = get_loss(model, X_batch, y_batch, w_batch)

    train_loss = np.mean(history[-40:])
    progress.set_description(f'Average batch loss = {train_loss:.6f}, test loss = {test_loss:.6f}')

In [ ]:
def print_weights(f, arr):
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            print("{0:0.5f}".format(arr[i][j]), ' ', file=f, sep="", end="")

def print_biases(f, arr):
    for i in range(arr.shape[0]):
            print("{0:0.5f}".format(arr[i]), ' ', file=f, sep="", end="")

with open('model.qnne', 'w') as f:
    print_weights(f, model.opening.linear1.weight.T.detach().numpy())
    print_weights(f, model.opening.linear2.weight.T.detach().numpy())
    print_weights(f, model.opening.linear3.weight.T.detach().numpy())
    print_biases(f, model.opening.linear1.bias.detach().numpy())
    print_biases(f, model.opening.linear2.bias.detach().numpy())
    print_biases(f, model.opening.linear3.bias.detach().numpy())